### Overview

This is how I merged multiple Excel files into one file. My files contained students' surveys and grades. Not all students completed all surveys. I was able to merge the files using participants' names because these were common to each file and there were no repeated names. The filenames and some actions (e.g., checking for data cleanliness) are specific to my project.

Once all data is in the merged file and each name has a unique ID associated with it (so participants cannot be identified using the data file alone)-- 
1. Create a master ID key--a separate file with the personally identifiable information and unique, randomly generated participant ID. 
2. Remove the personally identifiable information from the data file (e.g., name columns), but leave the unique, randomly generated IDs.

Notes about human subjects research: 
The unique, randomly generated participant ID should be specific to the research. If participants already have unique IDs that were not generated specifically for the resarch, these could be personally identifiable information. For example: A student ID is assigned to a student by an educational institution. It is personally identifiable information because a person from the institution with access to that ID would be able to identify the student even if that person is not a researcher on the project.

### Quick Tips/Reminders

* The # symbol is only necessary for code boxes. 
* You can't mix a bash command line with python code, even if it's just a python comment.
* Any boxes containing only markdown may be changed to markdown in the Jupyter menu and the # may be excluded. 
* To execute these cells in Jupyter notebook, hit shift enter.
* Normal python style has space on sides of equal sign, unless it's a parameter to a function.
* Use tab completion (start typing and hit tab) for it to guess what code you wanted to type.
* Go to cell-->current outputs-->clear to clear all the numbers on the left in the notebook.
* If have two cells want to merge in Jupyter Notebook, select top cell and hit shift m.
* If want to add a cell above, select the cell and hit a.
* If want to add a cell below, select the cell and it b.

In [1]:
#Start by importing useful python packages-

#Import pandas to use as the dataframe for our table structure
#Use pd as an alias for pandas
import pandas as pd

#Import operating system to navigate the file system
import os 

#Import random module to generate unique IDs
import random

### Navigate to files and read into pandas DataFrame.

* Use pwd to get the present working directory
* If needed, change directory using cd
* Can move up one directory using cd ../ 
 (In my case, it was a folder in the directory above my pwd)
* You can use tab completion so you don't have to type the whole name--Start typing the name of the directory you want & hit tab for the rest of the directory to show up.

In [ ]:
pwd

In [ ]:
cd ../Surveys\ used\ in\ master\ file

In [ ]:
ls

Read Excel tables into a separate pandas DataFrames.
Name the pandas DataFrames as follows:
 * Learning strategies surveys file-->strats
 * Beginning of semester survey file-->beginning
 * Midsemester survey file-->midsemester
 * Grades file-->grades
 * Mindsets Post, Then, Memory, Demographics file-->mindset_post
 * Mindsets Pre-Survey file-->mindset_pre

Tip to check out documentation:
* Start typing pd.rea (for pandas reading the excel file). 
* After strats = pd.read_excel, hit shift tab to show the signature and docstring of the Pandas read_excel method--the documentation for functions, classes, and modules. 
* Select the + symbol of the upper right corner of the signature/documentation that pops up to see all of it.
* It's not ovious, but io is the only parameter I'm using right now when I'm reading in this Excel file.
* io is the path (the filename in this case).
* Instead of doing shift tab, you could just add a question mark and it will bring up the same documentation.

In [ ]:
strats = pd.read_excel("Compiled-Learning Strategies6-master.xlsx")

In [ ]:
#Check to make sure have the right thing. Head gives us the first five lines of the Excel file.
strats.head()

Read in the remaining Excel files and check their headings.

In [ ]:
beginning = pd.read_excel("Beginning of Semester Survey-master.xlsx")

In [ ]:
beginning.head()

In [ ]:
midsemester = pd.read_excel('Evals-Midsemester-Results3-master.xlsx')

In [ ]:
midsemester.head()

In [ ]:
grades = pd.read_excel('Grades-Fall 2017c-master.xlsx')

In [ ]:
grades.head()

In [ ]:
mindset_post = pd.read_excel('Mindsets POST THEN MEMORY3-master.xlsx')

In [ ]:
mindset_post.head()

In [ ]:
mindset_pre = pd.read_excel('Mindsets Pre3-master.xlsx')

In [ ]:
mindset_pre.head()

In [ ]:
#I had some unnamed columns and wasn't sure why. 
#I did the following to get the number of filled cells for each column.
mindset_pre.count()

In [ ]:
#I fixed the mindset_pre file in Excel and then had pandas read the file again.
mindset_pre = pd.read_excel('Mindsets Pre3-master.xlsx')

In [ ]:
mindset_pre.head()

In [ ]:
#Create a Python dict (a dictionary) because we need something to a hold all our data tables.
#The key is a string on the left side of the colon, and the value is a pandas DataFrame in this dict.
data_tables = {
    'ls': strats,
    'grd': grades,
    'mid': midsemester,
    'beg': beginning,
    'mind1': mindset_pre,
    'mind2': mindset_post
}

### Merge data tables.

I need to join my files using a column from each data table. Once I choose that column, I'll set it as the index in each data table and then use it to join all my data tables. A good index would have an entry for every row of data and each entry would be unique. I believe the FullName column could be a good index, once lowercased and stripped of extraneous spaces, so I loop through each data table to check the FullName column as a potential index column. 

In [ ]:
#k=key (e.g, mid), v=value (e.g., midsemester)
#The for loop will loop through the key value pairs
#v.shape gives number of rows (and columns) in the DataFrame that v contains 
#potential_index.nunique() gives the number of unique values in potential_index column
#potential_index.count() gives number of values in potential_index column
#The output will have 6 rows, a row for each of my data tables.
#Each row will have the format: (k,(a,b),c,d)
#For FullName to be useful as an index, we want a=c=d for each DataFrame
#This code is just a check; we aren't actually creating anything new here

for k, v in data_tables.items():
    potential_index = v['FullName'].str.lower().str.strip()
    print(k, v.shape, potential_index.nunique(), potential_index.count())

The FullName column (lowercased and stripped) in each data table worked as a good index on which I could join my files, so I use it to merge my data files.

In [ ]:
#Initialize master_merge as an empty DataFrame
master_merge = pd.DataFrame()

#Call the items method by appending .items to the data_tables dict.
#The for loop will loop through the key value pairs
for k, v in data_tables.items():
    #Make a slightly modified copy of each of the DataFrames in the dict, setting the 
    #index in each DataFrame to full names that have been lowercased and stripped. 
    #The string method called lower makes entries lowercase.
    #The string method called strip removes leading and trailing whitespace from entries.
    data_to_merge = v.set_index(v['FullName'].str.lower().str.strip())
    
    #Use lambda for a throw away, quick use function
    #Default behavior of rename method is to return a copy, but we 
    #set inplace=True so it modifies the original object. 
    #For rename method to return a copy, you could set inplace=False or just leave off that part.
    #Example of what code below does:
    #All headers in strats DataFrame have ls_ prepended to the original header name
    data_to_merge.rename(columns=lambda x: k + '_' + str(x), inplace=True)
    
    #Joining files using outer b/c none of the files have all the names
    master_merge = master_merge.join(data_to_merge, how='outer')

In [ ]:
#Check headings and top rows
master_merge.head()

In [ ]:
#Check shape (This is a tuple (an ordered pair w/ # of rows, # of columns))
master_merge.shape

In [ ]:
#Add IDs column and randomly populate ID column with unique IDs.

#I picked a random starting value for my unique IDs as 600; there's nothing special about 600, just 
#that it's what I chose.
#Instead of writing the code below, I could have written: master_merge['IDs'] = random.sample(range(600, 800), 117)
#This would have been fine because I knew I needed 117 unique IDs and 600 to 800 is a big enough range for me to 
#randomly generate those unique IDs. 
#However, it wouldn't have been generalizable to future data files--I would have to replace those specific values.
master_merge['IDs'] = random.sample(range(600, 600 + master_merge.shape[0]), master_merge.shape[0])

In [ ]:
#Check headings and top rows (just to make sure it looks like it did what I wanted it to do)
master_merge.head()

In [ ]:
#Checking my present working directory
pwd

In [ ]:
#Saving my merged data tables to a master file
master_merge.to_excel('../Fall 2017 Master Data File.xlsx')

In [ ]:
#Saving my merged data tables to pickle b/c it serializes everything
#This is a great backup b/c sometimes you lose stuff important for Python when you save it as an Excel
#file, but you don't lose stuff when you save it as a pickle file
#The problem is you can't open it in Excel; you have to be in the Python environment to work on it
master_merge.to_pickle('../Fall 2017 Master Data File.pkl')